If your Dataset is stored on google drive, you have to connect the disk with notebook.

In [ ]:
import matplotlib.image as img
%matplotlib inline
import numpy as np
from collections import defaultdict
from shutil import copy
from shutil import copytree, rmtree
import collections
import os
import random
import cv2

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


Create .txt files with image filenames. Separate file for original and fake images.

In [ ]:
!ls /content/drive/Shareddrives/datasets/animals/dataset/original/*.jpg > /content/drive/Shareddrives/datasets/animals/originals.txt
!ls /content/drive/Shareddrives/datasets/animals/dataset/fake/fake*.jpg > /content/drive/Shareddrives/datasets/animals/fakes.txt

original_filenames_path = "/content/drive/Shareddrives/datasets/animals/originals.txt"
fake_filenames_path = "/content/drive/Shareddrives/datasets/animals/fakes.txt"

Paths to folders that include images. Current structure:
* dataset/
 *        -original/
 *       -fake/

In [ ]:
path = "/content/drive/Shareddrives/datasets/animals/"
original_path = path + "dataset/original"
fake_path = path + "dataset/fake"
original_train_path = path + "data/train/original"
fake_train_path = path + "data/train/fake"
original_test_path = path + "data/test/original"
fake_test_path = path + "data/test/fake"

Function that creates array of filenames

In [ ]:
def get_array_of_filenames(original_filenames_path, fake_filenames_path, main_path):
    ori_paths = []
    fake_paths = []
  

    with open(original_filenames_path, 'r') as file:
        ori_paths = [main_path + read.strip() for read in file.readlines()]

    with open(fake_filenames_path, 'r') as file:
        fake_paths = [main_path + read.strip() for read in file.readlines()]


    return ori_paths, fake_paths

In [ ]:
originals, fakes = get_array_of_filenames(original_filenames_path, fake_filenames_path, "")

The dataset has 16000 images so we are using a part of it. This function takes random images from the dataset.

In [ ]:
def train_test_big_split(originals, fakes):
    ori_len = len(originals)
    fakes_len = len(fakes)

    train_dataset_len = int(0.05 * ori_len)
    test_dataset_len = int(0.1 * train_dataset_len)
 
    ori_train_list = random.sample(range(0, ori_len), train_dataset_len)
    fakes_train_list = random.sample(range(0, fakes_len), train_dataset_len)
  
    ori_train_files = [originals[i] for i in ori_train_list]

    ori_test_files = []
    ori_test_list = []
    fakes_test_files = []
    fakes_test_list = []

    while len(ori_test_files) <= test_dataset_len :
        rnd = random.randint(0, ori_len)
        if (rnd not in ori_train_list and rnd not in ori_test_list):
            ori_test_files.append(originals[rnd])
        ori_test_list.append(rnd)
  
    fakes_train_files = [fakes[i] for i in fakes_train_list]
  
    while len(fakes_test_files) <= test_dataset_len :
        rnd = random.randint(0, fakes_len)
        if (rnd not in fakes_train_list and rnd not in fakes_test_list):
            fakes_test_files.append(fakes[rnd])
        fakes_test_list.append(rnd)
  
    return ori_train_files, ori_test_files, fakes_train_files, fakes_test_files

In [ ]:
ori_train, ori_test, fakes_train, fakes_test = train_test_big_split(originals, fakes)

This function creates file structure from our part of dataset that is neccessary for ImageDataGenerator.flow_from_directory() 
* data/
 *   -train/
    *        -original/
    *       -fake/
 *   -test/
    *        -original/
    *        -fake/
    
This may take a while because this method performs copying files.

In [ ]:
def prepare_files(ori_train, ori_test, fakes_train, fakes_test, src, ori_train_dest, fakes_train_dest, ori_test_dest, fakes_test_dest):
    if not os.path.exists(os.path.join(ori_train_dest)):
        os.makedirs(os.path.join(ori_train_dest))
    if not os.path.exists(os.path.join(fakes_train_dest)):
        os.makedirs(os.path.join(fakes_train_dest))
    if not os.path.exists(os.path.join(ori_test_dest)):
        os.makedirs(os.path.join(ori_test_dest))
    if not os.path.exists(os.path.join(fakes_test_dest)):
        os.makedirs(os.path.join(fakes_test_dest))

  
    for item in ori_train:
        copy(os.path.join(item), os.path.join(ori_train_dest, item.split("/")[-1]))
    print("Copying original train data, done")

    for item in ori_test:
        copy(os.path.join(item), os.path.join(ori_test_dest, item.split("/")[-1]))
    print("Copying original test data, done")

    for item in fakes_train:
        copy(os.path.join(item), os.path.join(fakes_train_dest, item.split("/")[-1]))
    print("Copying fake train data, done")

    for item in fakes_test:
        copy(os.path.join(item), os.path.join(fakes_test_dest, item.split("/")[-1]))
    print("Copying fake test data, done")

In [ ]:
prepare_files(ori_train, ori_test, fakes_train, fakes_test, original_path, original_train_path, fake_train_path, original_test_path, fake_test_path)

Copying original train data, done
Copying original test data, done
Copying fake train data, done
Copying fake test data, done


Now, we can move to the next script when proper analysis is performed.